#### Utils

In [1]:
seeds = [418, 627, 960, 426, 16, 523, 708, 541, 747, 897, 714, 127, 657, 662, 284, 595, 852, 734, 136, 394,
             321, 200, 502, 786, 817, 264, 929, 407, 515, 411]

In [2]:
import pandas as pd
from sklearn.metrics import roc_curve, accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import numpy as np
from typing import Dict
from statistics import mean, stdev

def get_df(path: str, seed) -> pd.DataFrame:
    df = pd.read_csv(path, header=None)
    df = df.rename(columns={0: "inference", 1: "ground_truth"})
    df["seed"] = seed
    df["ground_truth"] = df["ground_truth"].astype(float)
    return df

def get_best_treshold(ground_truths: np.ndarray, inferences: np.ndarray):
    fpr, tpr, tresholds = roc_curve(ground_truths, inferences)
    return tresholds[np.argmin(np.abs(tpr + fpr - 1))]

def get_predictions(inferences: np.ndarray, treshold):
    return np.where(inferences >= treshold, 1, 0)

def get_scores(results: Dict):
    datasets = ["deepmirtar", "miraw", "mirtarraw"]
    metrics = {"balanced_acc": balanced_accuracy_score, "f1": f1_score, "precision": precision_score, "recall": recall_score, "auc": roc_auc_score}
    for dataset in datasets:
        metric_scores = {key:[] for key in metrics.keys()}
        for seed in seeds:
            df = results[dataset]
            df = df[df["seed"]==seed]
            ground_truths = df["ground_truth"].to_numpy()
            inferences = df["inference"].to_numpy()
            prediction_treshold = get_best_treshold(ground_truths, inferences)
            predictions = get_predictions(inferences, prediction_treshold)
            for key, metric in metrics.items():
                score = metric(ground_truths, predictions)
                metric_scores[key].append(score)
        print_metric_scores(metric_scores, dataset)

def print_metric_scores(metric_scores, dataset, graph_layer=None):
    print("##########################")
    if graph_layer:
        print("Dataset: {}, Graph layer: {}".format(dataset, graph_layer))
    else:
        print("Dataset: {}".format(dataset))
    for key, scores in metric_scores.items():
        print("Metric: {}, mean: {}, min: {}, max: {}, std: {}".format(key, mean(scores), min(scores), max(scores), stdev(scores)))
        
def get_graph_scores(results: Dict):
    datasets = ["deepmirtar", "miraw", "mirtarraw"]
    metrics = {"balanced_acc": balanced_accuracy_score, "f1": f1_score, "precision": precision_score, "recall": recall_score, "auc": roc_auc_score}
    gnn_layer_type = ["GCN", "GAT", "GRAPHSAGE"]
    for dataset in datasets:
        for gnn_layer in gnn_layer_type:
            metric_scores = {key:[] for key in metrics.keys()}
            for seed in seeds:
                df = results[dataset][gnn_layer]
                df = df[df["seed"]==seed]
                ground_truths = df["ground_truth"].to_numpy()
                inferences = df["inference"].to_numpy()
                prediction_treshold = get_best_treshold(ground_truths, inferences)
                predictions = get_predictions(inferences, prediction_treshold)
                for key, metric in metrics.items():
                    score = metric(ground_truths, predictions)
                    metric_scores[key].append(score)
            print_metric_scores(metric_scores, dataset, gnn_layer)

### miraw

In [3]:
configs = [
        (128, "deepmirtar", 0.001),
        (64, "miraw", 0.001),
        (128, "mirtarraw", 0.001),
    ]

In [4]:
results = {}
for config in configs:
    df_list = []
    batch_size, dataset, lr = config
    for seed in seeds:
        filename = "../../experiments/miraw/results/miraw_ann_{}_config_{}_{}_{}.csv".format(dataset, batch_size, seed, lr)
        df = get_df(filename, seed)
        df_list.append(df)
    concatenated_dfs = pd.concat(df_list, axis=0)
    results[dataset] = concatenated_dfs

In [5]:
get_scores(results)

##########################
Dataset: deepmirtar
Metric: balanced_acc, mean: 0.9022384999713331, min: 0.8798661064657183, max: 0.9270610365181293, std: 0.012371089815869247
Metric: f1, mean: 0.9024761293232006, min: 0.8763250883392227, max: 0.9257641921397379, std: 0.01342717196645826
Metric: precision, mean: 0.9024758523794282, min: 0.8717047451669596, max: 0.9250814332247557, std: 0.015528950326991852
Metric: recall, mean: 0.9024977387827366, min: 0.8809946714031972, max: 0.9281961471103327, std: 0.011787962198240558
Metric: auc, mean: 0.9022384999713331, min: 0.8798661064657183, max: 0.9270610365181293, std: 0.012371089815869228
##########################
Dataset: miraw
Metric: balanced_acc, mean: 0.9378301597007878, min: 0.9319091452698178, max: 0.9455263273475778, std: 0.0031461380635202665
Metric: f1, mean: 0.9382810025008321, min: 0.9321053758834869, max: 0.9456244689889549, std: 0.0031787374010597857
Metric: precision, mean: 0.9387382037108534, min: 0.9317062727467351, max: 0.946

### deepmirtar

In [6]:
configs = [
        (512, "deepmirtar", 0.001),
        (512, "miraw", 0.001),
        (256, "mirtarraw", 0.001),
    ]

In [7]:
results = {}
for config in configs:
    df_list = []
    batch_size, dataset, lr = config
    for seed in seeds:
        filename = "../../experiments/deepmirtar/results/deepmirtar_ann_{}_config_{}_{}_{}.csv".format(dataset, batch_size, seed, lr)
        df = get_df(filename, seed)
        df_list.append(df)
    concatenated_dfs = pd.concat(df_list, axis=0)
    results[dataset] = concatenated_dfs

In [8]:
get_scores(results)

##########################
Dataset: deepmirtar
Metric: balanced_acc, mean: 0.8153637961403988, min: 0.7948390559205483, max: 0.8429279614032604, std: 0.011130223476319341
Metric: f1, mean: 0.8159083393029651, min: 0.7964444444444445, max: 0.8460891505466779, std: 0.01133108582223124
Metric: precision, mean: 0.8165716806466606, min: 0.7901234567901234, max: 0.8496621621621622, std: 0.013031275876639863
Metric: recall, mean: 0.8152904260000569, min: 0.7953020134228188, max: 0.8425460636515912, std: 0.011136657216198468
Metric: auc, mean: 0.8153637961403988, min: 0.7948390559205483, max: 0.8429279614032603, std: 0.011130223476319317
##########################
Dataset: miraw
Metric: balanced_acc, mean: 0.87501129417586, min: 0.8693473337779309, max: 0.8846685406206235, std: 0.00370728561662137
Metric: f1, mean: 0.8758670363825928, min: 0.8698351530721473, max: 0.8848768054375532, std: 0.0036435589216141486
Metric: precision, mean: 0.8767010081382495, min: 0.8690909090909091, max: 0.8850647

### mitar

In [9]:
configs = [
        (16, "deepmirtar", 0.001),
        (64, "miraw", 0.001),
        (64, "mirtarraw", 0.001),
    ]

In [10]:
results = {}
for config in configs:
    df_list = []
    batch_size, dataset, lr = config
    for seed in seeds:
        filename = "../../experiments/mitar/results/mitar_net_{}_config_{}_{}_{}.csv".format(dataset, batch_size, seed, lr)
        df = get_df(filename, seed)
        df_list.append(df)
    concatenated_dfs = pd.concat(df_list, axis=0)
    results[dataset] = concatenated_dfs

In [11]:
get_scores(results)

##########################
Dataset: deepmirtar
Metric: balanced_acc, mean: 0.9274428608712387, min: 0.8944088203543008, max: 0.9502275514053952, std: 0.011969235007181955
Metric: f1, mean: 0.9277094842802632, min: 0.897071129707113, max: 0.9506802721088435, std: 0.01173023783166883
Metric: precision, mean: 0.9279809210795459, min: 0.8993288590604027, max: 0.9522998296422487, std: 0.011950042354267554
Metric: recall, mean: 0.9274481229321292, min: 0.8948247078464107, max: 0.9490662139219015, std: 0.011909313053262352
Metric: auc, mean: 0.9274428608712387, min: 0.8944088203543008, max: 0.950227551405395, std: 0.01196923500718194
##########################
Dataset: miraw
Metric: balanced_acc, mean: 0.9393835730591168, min: 0.9315852091439464, max: 0.9444615165129804, std: 0.0035676128967912593
Metric: f1, mean: 0.9398157333683317, min: 0.9322447248436009, max: 0.94529448752882, std: 0.0037150969743315132
Metric: precision, mean: 0.9402676854276337, min: 0.9325413661434027, max: 0.94608768

### graphtar

In [12]:
configs = [
        ("GCN", 128, "deepmirtar", 128, 3, "MAX", 512, 2, 0.4, 0.001),
        ("GCN", 64, "miraw", 128, 3, "MAX", 512, 2, 0.4, 0.001),
        ("GCN", 64, "mirtarraw", 128, 3, "MAX", 512, 2, 0.4, 0.001),
        ("GAT",128, "deepmirtar", 256, 5, "ADD", 128, 2, 0.4, 0.001),
        ("GAT", 512, "miraw", 256, 5, "ADD", 128, 2, 0.4, 0.001),
        ("GAT", 512, "mirtarraw", 256, 5, "ADD", 128, 2, 0.4, 0.001),
        ("GRAPHSAGE", 128, "deepmirtar", 256, 5, "ADD", 256, 3, 0.4, 0.001),
        ("GRAPHSAGE", 32, "miraw", 256, 5, "ADD", 256, 3, 0.4, 0.001),
        ("GRAPHSAGE", 256, "mirtarraw", 256, 5, "ADD", 256, 3, 0.4, 0.001),
    ]

In [13]:
results = {}
for config in configs:
    df_list = []
    gnn_layer_type, batch_size, dataset_type, graph_layer_size, n_gnn_layers, global_pooling, fc_layer_size, n_fc_layers, dropout_rate, lr = config
    for seed in seeds:
        filename = "../../experiments/graphtar/results/{}_{}_{}.csv".format(gnn_layer_type, dataset_type, seed)
        df = get_df(filename, seed)
        df_list.append(df)
    concatenated_dfs = pd.concat(df_list, axis=0)
    if results.get(dataset, None) is None:
        results[dataset_type] = {}
    results[dataset_type][gnn_layer_type] = concatenated_dfs

In [14]:
get_graph_scores(results)

##########################
Dataset: deepmirtar, Graph layer: GCN
Metric: balanced_acc, mean: 0.904285743881202, min: 0.8678138558762498, max: 0.9338432184396159, std: 0.013823708520151518
Metric: f1, mean: 0.9046144564522566, min: 0.869047619047619, max: 0.9355648535564853, std: 0.01403123187542384
Metric: precision, mean: 0.9046450320458003, min: 0.8705281090289608, max: 0.9347826086956522, std: 0.014383583930276276
Metric: recall, mean: 0.9045960529857009, min: 0.8675721561969439, max: 0.9363484087102177, std: 0.014080622984177291
Metric: auc, mean: 0.904285743881202, min: 0.8678138558762498, max: 0.933843218439616, std: 0.013823708520151556
##########################
Dataset: deepmirtar, Graph layer: GAT
Metric: balanced_acc, mean: 0.9221780280890802, min: 0.878960943257185, max: 0.949346202845212, std: 0.014594619586487109
Metric: f1, mean: 0.9223326015161759, min: 0.8804071246819339, max: 0.9505448449287511, std: 0.015046976758207795
Metric: precision, mean: 0.9226114047326192, mi